In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np

import pygsti
from pygsti.extras import circuit

/Users/tjproct/pyGSTi/packages/pygsti/tools/matrixtools.py:23: UserWarning: Could not import Cython extension - falling back to slower pure-python routines
  _warnings.warn("Could not import Cython extension - falling back to slower pure-python routines")


In [2]:
# The most basic initialization of a BasicGateSet object. It is essentially empty
bgs = circuit.BasicGateSet(n=2)

In [3]:
# A list of standard hard-coded gate labels
gllist = ['I', 'H','P','CNOT']

# A dictionary of unitaries that do not need to be already known to the code
unitaries={'T' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}

# The number of qubits
n = 10

# The availiability of gates that are not available to all qubit / qubit pairs
availability = {}

# Let's make a the CNOT gate be connected in a directed ring.
a = np.zeros((n,n),int)
for i in range(0,n-1):
    a[i,i+1] = 1
a[n-1,0] = 1

availability={'CNOT':a}

# Specifies whether Clifford-only quantities should be populated for the gates that are Clifford
clifford = True

BGS = circuit.BasicGateSet(n=n,gllist=gllist,unitaries=unitaries,clifford=clifford)

In [4]:
BGS.smatrix

{u'CNOT': array([[1, 0, 0, 0],
        [1, 1, 0, 0],
        [0, 0, 1, 1],
        [0, 0, 0, 1]]), u'H': array([[0, 1],
        [1, 0]]), u'I': array([[1, 0],
        [0, 1]]), u'P': array([[1, 0],
        [1, 1]]), u'T': None}

In [5]:
# We can also populate a DeviceSpec using the same things. When the device spec is made it populates some
# extra things that are useful for circuit compiling.

# If this is true, if creates compilations for a set of "standard" Clifford gates
construct_std_compilations=True

# Todo : the device spec will give an error when the idle is not in the gateset. This is ok, but it should 
# do an assert.

ds = circuit.DeviceSpec(n, gllist, unitaries, clifford=True, availability=availability,
                        construct_std_compilations=construct_std_compilations, verbosity=0)

In [6]:
# This is a CompilationLibraries object, which holds compilations for, e.g., CNOT between 1 and 4
compilations = ds.compilations

# There are two types of compilation, stored under compilations.paulieq and compilations.absolute.
# The latter is a absolute compilation, i.e., the circuit implements the claimed gate. The former only
# implements the claimed gate up to Paulis.

c = ds.compilations.paulieq[circuit.Gate('CNOT',(1,4))]

print(c)

c = ds.compilations.absolute[circuit.Gate('X',1)]

print(c)

Qubit 0 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 1 ---|C2|-|  |-|  |-|  |-|C2|-|  |-|  |-|  |---
Qubit 2 ---|T1|-|C3|-|  |-|C3|-|T1|-|C3|-|  |-|C3|---
Qubit 3 ---|  |-|T2|-|C4|-|T2|-|  |-|T2|-|C4|-|T2|---
Qubit 4 ---|  |-|  |-|T3|-|  |-|  |-|  |-|T3|-|  |---
Qubit 5 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 6 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 7 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 8 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 9 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---

Qubit 0 ---|  |-|  |-|  |-|  |---
Qubit 1 ---|H |-|P |-|P |-|H |---
Qubit 2 ---|  |-|  |-|  |-|  |---
Qubit 3 ---|  |-|  |-|  |-|  |---
Qubit 4 ---|  |-|  |-|  |-|  |---
Qubit 5 ---|  |-|  |-|  |-|  |---
Qubit 6 ---|  |-|  |-|  |-|  |---
Qubit 7 ---|  |-|  |-|  |-|  |---
Qubit 8 ---|  |-|  |-|  |-|  |---
Qubit 9 ---|  |-|  |-|  |-|  |---



In [7]:
# A matrix giving the shorest path between qubits
print(ds.shortestpath)

# A matrix giving the distance between qubits
print(ds.distance)

[[-9999     0     1     2     3     4     7     8     9     0]
 [    1 -9999     1     2     3     4     5     8     9     0]
 [    1     2 -9999     2     3     4     5     6     9     0]
 [    1     2     3 -9999     3     4     5     6     7     0]
 [    1     2     3     4 -9999     4     5     6     7     0]
 [    1     2     3     4     5 -9999     5     6     7     8]
 [    9     2     3     4     5     6 -9999     6     7     8]
 [    9     0     3     4     5     6     7 -9999     7     8]
 [    9     0     1     4     5     6     7     8 -9999     8]
 [    9     0     1     2     3     6     7     8     9 -9999]]
[[ 0.  1.  2.  3.  4.  5.  4.  3.  2.  1.]
 [ 1.  0.  1.  2.  3.  4.  5.  4.  3.  2.]
 [ 2.  1.  0.  1.  2.  3.  4.  5.  4.  3.]
 [ 3.  2.  1.  0.  1.  2.  3.  4.  5.  4.]
 [ 4.  3.  2.  1.  0.  1.  2.  3.  4.  5.]
 [ 5.  4.  3.  2.  1.  0.  1.  2.  3.  4.]
 [ 4.  5.  4.  3.  2.  1.  0.  1.  2.  3.]
 [ 3.  4.  5.  4.  3.  2.  1.  0.  1.  2.]
 [ 2.  3.  4.  5.  4.  3.

In [8]:
print(ds.compilations.absolute[ds.compilations.absolute.keys()[0]])

Qubit 0 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 1 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 2 ---|P |-|P |-|H |-|P |-|P |-|H |---
Qubit 3 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 4 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 5 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 6 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 7 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 8 ---|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 9 ---|  |-|  |-|  |-|  |-|  |-|  |---



In [9]:
# Device specs can be given "GateSetModel" that is stored in the .models dictionary
ds.models["Null model"] = circuit.GateSetModel(operators=None,n=n,mtype='purestate')

In [10]:
# Let's create a new DeviceSpec on less qubits, with full connectivity
gllist = ['I', 'H','P','CPHASE']

# The number of qubits
n = 4

ds2 = circuit.DeviceSpec(n, gllist,  clifford=True, verbosity=1)

Generating a pauli-equivalence compilation for the H gate...Complete.
Generating a pauli-equivalence compilation for the P gate...Complete.
Generating a pauli-equivalence compilation for the PH gate...Complete.
Generating a pauli-equivalence compilation for the HP gate...Complete.
Generating a pauli-equivalence compilation for the HPH gate...Complete.
Generating an absolute compilation for the I gate...Complete.
Generating an absolute compilation for the X gate...Complete.
Generating an absolute compilation for the Y gate...Complete.
Generating an absolute compilation for the Z gate...Complete.
Creating a circuit to implement H up to Pauli gates on qubit 0...Complete.
Creating a circuit to implement P up to Pauli gates on qubit 0...Complete.
Creating a circuit to implement PH up to Pauli gates on qubit 0...Complete.
Creating a circuit to implement HP up to Pauli gates on qubit 0...Complete.
Creating a circuit to implement HPH up to Pauli gates on qubit 0...Complete.
Creating a circuit 

In [11]:
# We can also create less trivial examples, which then allow us to do circuit simulations (the model
# above fairly obviously won't work if we hand it to a simulator).

# Lets construct a model which has perfect gates. Note that the code below wouldn't be correct
# for a device with non-symmetric gates -- in that case care has to be taken to input the 
# unitaries correctly.
operators = {}
for gate in ds2.allgates:
    operators[gate] = ds2.gateset.unitaries[gate.label]
    
ds2.models['Target'] = circuit.GateSetModel(operators,n,mtype='purestate')

In [12]:
# Let's pick a random Clifford, and the inverse of that Clifford, and create an identity circuit
s, p = circuit.random_clifford(4)
sin, pin = circuit.inverse_clifford(s,p)

# Creates the circuit, over the gates in ds2.
c = circuit.compile_clifford(s, p, ds=ds2)
c.append_circuit(circuit.compile_clifford(sin, pin, ds=ds2))

#print(c)

In [17]:
# As we can see, the output is (0,0,0,0) with probability 1. (when no input is given to the simulators, the input
# is taken to be (0,0,0,0)).

simout = c.simulate(ds2.models['Target'])
print(simout[0,0,0,0])
x = 0
for key in list(simout.keys()):
    x += simout[key]
x = x - simout[0,0,0,0]
print(x)

1.0
0.0
